In [10]:
import pandas as pd
import os
import shutil
from Bio import SeqIO
from itertools import combinations
import subprocess

In [11]:
work_path='/Users/jiachenghao/Desktop/shenzhen_test/'
ARG_name='adeF'
inputname='3000777_2'

In [12]:
if os.path.exists(work_path+'Extract'):
    Extract_fasta_file=work_path+'Extract/'
else:
    os.mkdir(work_path+'Extract/')
    Extract_fasta_file=work_path+'Extract/'


if os.path.exists(work_path+'Motifi_Extract_Fasta'):
    Motifi=work_path+'Motifi_Extract_Fasta/'
else:
    os.mkdir(work_path+'Motifi_Extract_Fasta/')
    Motifi=work_path+'Motifi_Extract_Fasta/'


if os.path.exists(work_path+ARG_name+'_list'):
    List_path=work_path+ARG_name+'_list/'
else:
    os.mkdir(work_path+ARG_name+'_list/')
    List_path=work_path+ARG_name+'_list/'


if os.path.exists(work_path+'rgi_file/'):
    file_path=work_path+'rgi_file/'
else:
    os.mkdir(work_path+'rgi_file/')
    file_path=work_path+'rgi_file/'


if os.path.exists(work_path+'fasta_file/'):
    fasta_path=work_path+'fasta_file/'
else:
    os.mkdir(work_path+'fasta_file/')
    fasta_path=work_path+'fasta_file/'

inputname=work_path+inputname+'/'   

In [13]:
def modify_contig_names(input_file, output_file, prefix):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            if line.startswith('>'):
                new_contig_name = f">{prefix}\n"
                f_out.write(new_contig_name)
            else:
                f_out.write(line)

def merge_fasta_files(input_folder, output_file):
    with open(output_file, 'w') as out_handle:
        for filename in os.listdir(input_folder):
            if filename.endswith('.fna'):
                file_path = os.path.join(input_folder, filename)
                for record in SeqIO.parse(file_path, 'fasta'):
                    SeqIO.write(record, out_handle, 'fasta')

#定义函数，用于截取反向互补
def rev(seq):
    base_trans = {'A':'T', 'C':'G', 'T':'A', 'G':'C', 'a':'t', 'c':'g', 't':'a', 'g':'c'}
    rev_seq = list(reversed(seq))
    rev_seq_list = [base_trans[k] for k in rev_seq]
    rev_seq = ''.join(rev_seq_list)
    return(rev_seq)

########################
def extract_fasta(input, list, output):
    seq_file = {}
    with open(input, 'r') as input_fasta:
        for line in input_fasta:
            line = line.strip()
            if line[0] == '>':
                seq_id = line.split()[0]
                seq_file[seq_id] = ''
            else:
                seq_file[seq_id] += line
    input_fasta.close()

    list_dict = {}
    with open(list, 'r') as list_file:
        for line in list_file:
            if line.strip():
                line = line.strip().split('\t')
                list_dict[line[0]] = [line[1], int(line[2]) - 1, int(line[3]), line[4]]
    list_file.close()

    output_fasta = open(output, 'w')
    for key,value in list_dict.items():
        print('>' + key, file = output_fasta)
        seq = seq_file['>' + value[0]][value[1]:value[2]]
        if value[3] == '+':
            print(seq, file = output_fasta)
        elif value[3] == '-':
            seq = rev(seq)
            print(seq, file = output_fasta)
    output_fasta.close()
    
##########################
    
def remove_duplicates(strings):
    seen_pairs = set()    #########
    result = []
    for string in strings:
        parts = string.split('-')
        pair = tuple(sorted(parts))
        if pair not in seen_pairs:
            seen_pairs.add(pair)
            result.append(string)
    return result

def remove_duplicates2(strings):
    result = []
    for string in strings:
        parts = string.split('-')
        if len(parts) == 2 and parts[0] != parts[1]:
            result.append(string)
    return result


Process_File

In [7]:
file_path

'/Users/jiachenghao/Desktop/shenzhen_test/rgi_file/'

In [14]:
inputname_list=os.listdir(inputname)
inputname_list_fna=[x for x in inputname_list if x.endswith('.fna')]
inputname_list_txt=[x for x in inputname_list if x.endswith('.txt')]
if len(inputname_list_fna) == len(inputname_list_txt):
    print('Start process '+ str(len(inputname_list_txt)) + ' files' )
for i in inputname_list_txt:
    shutil.copy(inputname+i,file_path)
for i in inputname_list_fna:
    shutil.copy(inputname+i,fasta_path)
#####txtfile+filename####
for i in inputname_list_txt:
    i_file=pd.read_csv(file_path+i,sep='\t')
    pending_item_length=i_file.shape[0]
    i_name=i.replace('txt','fna')
    pending_item=[i_name]*pending_item_length
    i_file['#FILE']=pending_item
    i_file.to_csv(file_path+i,sep='\t')
###check_process
# for i in inputname_list_txt:
#     check_process=pd.read_csv(file_path+i,sep='\t')
#     check_process=check_process[check_process['Best_Hit_ARO']==ARG_name]
#     if check_process.shape[0] != 1:
#         os.remove(file_path+i)
# print('Check_process_complete, '+str(len(os.listdir(file_path)))+ ' remain')

merged_txt=work_path+'All_txt.txt'

with open(merged_txt, 'w') as outfile:
    for filename in os.listdir(file_path):
        if filename.endswith('.txt'):
            temp_file_path = os.path.join(file_path, filename)
            with open(temp_file_path, 'r') as infile:
                outfile.write(infile.read())


file=pd.read_csv(merged_txt,sep='\t')
file=file[file['Best_Hit_ARO']==ARG_name]
file=file.drop("Unnamed: 0", axis=1)
file.to_csv(work_path+'Target_ARG_summary.csv',index=False)
file=pd.read_csv('Target_ARG_summary.csv')
file_length=file.shape[0]
os.remove(merged_txt)

Start process 17 files


#############################################################

Start Extract

In [15]:
counter=0
while counter < file_length:
    txt_out=[]
    temp_file=file[file.index==counter]
    fasta_list_filename=temp_file['#FILE'].to_list()[0]
    ARG_name=temp_file['Best_Hit_ARO'].to_list()[0]

    SEQUENCE=temp_file['Contig'].to_list()[0]
    last_underscore_index=SEQUENCE.rfind('_')
    SEQUENCE=SEQUENCE[:last_underscore_index]
    START=temp_file['Start'].to_list()[0]
    START=int(START)
    if START-5000 < 0:
        START=1     ###########bug 1
        START=str(START)
    else:
        START=START-5000
        START=str(START)
    END=temp_file['Stop'].to_list()[0]
    END=int(END)
    END=END+5000
    END=str(END)
    STRAND=temp_file['Orientation'].to_list()[0]
    txt_out.append(SEQUENCE)
    txt_out.append(START)
    txt_out.append(END)
    txt_out.append(STRAND)

    if len(txt_out)==4:
        txt_out=pd.DataFrame(txt_out)
        txt_out=txt_out.T
        txt_out.to_csv(List_path+fasta_list_filename+'_'+str(counter),sep='\t',header=False)
    counter+=1

In [ ]:
List_path_file=os.listdir(List_path)
for i in List_path_file:
    last_underscore_index=i.rfind('_')
    real_fasta_name=i[:last_underscore_index]
    print(real_fasta_name)

    print('Extracting '+i)
    input_fasta=fasta_path+real_fasta_name
    list_file=List_path+i
    output=Extract_fasta_file+i
    extract_fasta(input_fasta,list_file,output)

fastas=os.listdir(Extract_fasta_file)
for i in fastas:
    contig_name=i.replace('.fna','')
    modify_contig_names(Extract_fasta_file+i, Motifi+i,contig_name)


# output_file=ARG_name+' merged.fasta'
# merge_fasta_files(Motifi, output_file)

In [27]:
########多一步 Mutil Fasta Files合并########
Motified_fastas=os.listdir(Motifi)
temp_Motified=[]
for i in Motified_fastas:
    last_underscore_index=i.rfind('_')
    temp_temp_Motified=i[:last_underscore_index]
    temp_Motified.append(temp_temp_Motified)

set_temp_Motified=list(set(temp_Motified))
for i in set_temp_Motified:

    temp_count=temp_Motified.count(i)

    if temp_count > 1:
        temp_Motified_file_list=[x for x in Motified_fastas if x.startswith(i)]

        with open(Motifi+i, 'w') as outfile:
            for ii in temp_Motified_file_list:
                with open(Motifi+ii, 'r') as infile:
                    for line in infile:
                        outfile.write(line)
    else:
        temp_Motified_file_list=[x for x in Motified_fastas if x.startswith(i)][0]
        os.rename(Motifi+temp_Motified_file_list,Motifi+i)

In [28]:
Motified_fastas=os.listdir(Motifi)

for i in Motified_fastas:
    ii = os.path.join(Motifi, i)
    if os.path.isfile(ii) and not i.endswith(".fna"):
        os.remove(ii)

In [29]:
output_file=ARG_name+' merged.fasta'
merge_fasta_files(Motifi, output_file)

构建Blastn文件结构

In [30]:
if os.path.exists(work_path+'blastn'):
    blastn_path=work_path+'blastn'
else:
    os.mkdir(work_path+'blastn')
    blastn_path=work_path+'blastn'

if os.path.exists(work_path+output_file):
    shutil.move(work_path+output_file,blastn_path)
else:
    print(output_file, " Missing")

if os.path.exists(Motifi):
    shutil.move(Motifi,blastn_path)
else:
    print("Motifi folder Missing")

if os.path.exists(work_path+'blastn.sh'):
    shutil.copy(work_path+'blastn.sh',blastn_path)
else:
    print("Blatn Script Missing")

os.mkdir(blastn_path+'/OUT')
os.mkdir(blastn_path+'/merged_DB')
os.rename(blastn_path+'/'+output_file, blastn_path+'/'+'merged.fasta')
shutil.move(blastn_path+'/merged.fasta',blastn_path+'/merged_DB')

'/Users/jiachenghao/Desktop/shenzhen_test/blastn/merged_DB/merged.fasta'

构建blastn索引

In [31]:
working_directory=blastn_path+'/merged_DB'
command = ["makeblastdb", "-in", "merged.fasta", "-dbtype", "nucl", "-out", "merged", "-parse_seqids"]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=working_directory)
if result.returncode == 0:
    print("Blastn DB Run Success")
    print(result.stdout.decode())
else:
    print("Blastn DB Run Failed")
    print(result.stderr.decode())


Blastn DB Run Success


Building a new DB, current time: 05/05/2024 10:40:46
New DB name:   /Users/jiachenghao/Desktop/shenzhen_test/blastn/merged_DB/merged
New DB title:  merged.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 36 sequences in 0.00363398 seconds.





运行Blastn

In [32]:
working_directory=blastn_path
command = ["sh", "blastn.sh"]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=working_directory)

# 输出命令执行结果
if result.returncode == 0:
    print("Blastn Run Complete")
    print(result.stdout.decode())
else:
    print("Blastn Run Failed")
    print(result.stderr.decode())

计算HGT Frequent Percentage

In [ ]:
working_directory=blastn_path+'/OUT'
command = ["cat", "*.txt", ">", "Final_blastn_merged.txt"]
result = subprocess.run(" ".join(command), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=working_directory)

if result.returncode == 0:
    print("File Combined")
    print(result.stdout.decode())
else:
    print("File Combined Failed")
    print(result.stderr.decode())

shutil.move(blastn_path+'/OUT/Final_blastn_merged.txt',work_path)

######下面重写

In [2]:
file=pd.read_csv('Final_blastn_merged.txt',sep='\t',header=None)
column=['name1','name2','name3','name4','name5','name6','name7','name8','name9','name10',
        'name11','name12','name13','name14']
file.columns=column
file_length2=file.shape[0]
x_col=['-']*file_length2
file['x']=x_col
file['y']=file.apply(lambda row: ''.join(row[['name1', 'x', 'name2']]), axis=1)

file_name1=list(set(file['name1'].to_list()))
print(len(file_name1))

17


In [3]:
All_Y=[]
All_verify=[]
for i in file_name1:
    temp_file=file[file['name1']==i]

    new_index=i+'-'+i
    temp_file2=temp_file[temp_file['y']==new_index]
    temp_file2_name4=temp_file2['name4'].to_list()
    max_value = max(temp_file2_name4)

    temp_file3=temp_file[temp_file['name4']==max_value]
    temp_file3=temp_file3[temp_file3['name3'] >=  99]
    
    ready_to_depulicate=temp_file3['y'].to_list()
    verfy=temp_file3['name3'].to_list()
    for ii in ready_to_depulicate:
        All_Y.append(ii)
    for iii in verfy:
        All_verify.append(iii)

In [6]:
filtered_1=remove_duplicates(All_Y)
filtered_2=remove_duplicates2(filtered_1)

In [11]:
temp_out=pd.DataFrame(filtered_2)
temp_out.to_csv('3002972_162.csv',index=False)

In [ ]:
# final_all_all=file_length*file_length    #######Bug2
final_all_count=len(filtered_2)
print(final_all_count)
final_all_all=(1+(file_length-1))*((file_length-1)/2)
print(final_all_all) 
HGT_Frequency=final_all_count/final_all_all
HGT_Frequency=round(HGT_Frequency,2)
print('HGT Frequency for ', ARG_name, 'is:', HGT_Frequency)

Deleting Files

In [ ]:
shutil.rmtree(work_path+'blastn')
shutil.rmtree(work_path+'Extract')
shutil.rmtree(work_path+'fasta_file')
shutil.rmtree(work_path+'rgi_file')
shutil.rmtree(work_path+ARG_name+'_list/')
os.remove(work_path+'Final_blastn_merged.txt')
os.remove(work_path+'Target_ARG_summary.csv')